<h1 align='center'>Construindo bons conjuntos de dados de treinamento - pré-processamento de dados</h1>
<p align= 'center'><img src=https://sloboda-studio.com/wp-content/uploads/2020/08/Group-192.jpg></p>
<p align= 'justify'>A qualidade dos dados e a quantidade de informações úteis que eles contêm são fatores-chave que determinam o quão bem um algoritmo de aprendizado de máquina pode aprender. Portanto, é absolutamente crítico garantir que examinemos e pré-processemos um conjunto de dados antes de alimentá-lo a um algoritmo de aprendizado.</p>

<h3>Lidando com dados perdidos</h3>
<p align= 'justify'>Não é incomum em aplicações do mundo real que nossos exemplos de treinamento estejam faltando um ou mais valores por vários motivos. Pode ter havido um erro no processo de coleta de dados, certas medidas podem não ser aplicáveis ou campos específicos podem ter sido simplesmente deixados em branco em uma pesquisa, por exemplo. Normalmente, vemos valores ausentes como espaços em branco em nossa tabela de dados ou como strings de espaço reservado, como <i>NaN</i>, que significa "não é um número" ou <i>NULL</i> (um indicador comumente usado de valores desconhecidos em bancos de dados relacionais). Infelizmente, a maioria das ferramentas computacionais são incapazes de lidar com esses valores ausentes ou produzirão resultados imprevisíveis se simplesmente os ignorarmos. Portanto, é crucial que cuidemos desses valores ausentes antes de prosseguirmos com análises adicionais.</p>

<h3>Identificando valores ausentes em dados tabulares</h3>

In [20]:
import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


<p align='justify'>Para um DataFrame maior, pode ser tedioso procurar manualmente os valores ausentes; neste caso, podemos usar o método <i>isnull</i> para retornar um DataFrame com valores booleanos que indicam se uma célula contém um valor numérico <i>(False)</i> ou se faltam dados <i>(True)</i>. Usando o método <i>sum</i>, podemos retornar o número de valores ausentes por coluna da seguinte forma:</p>

In [21]:
# O somatório das colunas que possuem dados NaN ou Nulos.
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

<h4>Eliminando exemplos de treinamento ou recursos com valores ausentes</h4>

In [22]:
# Removendo todas as linhas que tem Nan
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [23]:
# Removendo todas as colunas que tem Nan
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [24]:
# Removendo as linhas se TODAS forem NaN.
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [25]:
# Removendo as linhas se tiver pelo menos que 4 valores reais.
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [26]:
# Removendo as linhas aonde aparece NaN em uma coluna específica, no caso "C".
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


<p align='justify'>Embora a remoção de dados ausentes pareça ser uma abordagem conveniente, ela também apresenta algumas desvantagens; por exemplo, podemos acabar removendo muitas amostras, o que impossibilitará uma análise confiável. Ou, se removermos muitas colunas de <i>features</i>, corremos o risco de perder informações valiosas que nosso classificador precisaria para discriminar entre as classes. Uma das alternativas mais usadas para lidar com valores ausentes é chamada de <b>Técnica de Interpolação</b>.</p>

<h3>Como imputar valores ausentes</h3>
<p align='justify'>Como foi dito anteriomente, excluir as linhas ou colunas pode comprometer dados valiosos, no conjunto como um todo. Como alteranativa, podemos usar diferentes técnicas de interpolação para estimar os valores ausentes dos outros exemplos de treinamento em nosso conjunto de dados. Uma das técnicas de interpolação mais comuns é a imputação média, onde simplesmente substituímos o valor ausente pelo valor médio de toda a coluna de <i>features</i>. Uma maneira conveniente de conseguir isso é usando a classe <i>SimpleImputer</i> do <i>scikit-learn</i>, conforme mostrado no código a seguir:</p>

In [27]:
# Conjunto de dados
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [28]:
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean') # Utilizando a MÉDIA. Poderíamos utilizar: "median"(Mediana) ou 'most_frequent'(Mais Frequente)

# O "most_frequent" é usando para colunas categóricas: cores, sexo, tamanho (P,M,G), entre outras.


imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [29]:
# Alternativamente poderíamos usar o Pandas "fillna" para prover o mesmo recurso.

df.fillna(df.mean()) 

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


<h3>Entendendo a API do estimador scikit-learn</h3>
<p align='justify'>Anteriormente, usamos a classe <i>SimpleImputer</i> do <i>scikit-learn</i> para imputar valores ausentes em nosso conjunto de dados. A classe <i>SimpleImputer</i> pertence às chamadas classes de transformadores no <i>scikit-learn</i>, que são usadas para transformação de dados. Os dois métodos essenciais desses estimadores são o ajuste e a transformação. O método <i>fit</i> é usado para aprender os parâmetros dos dados de treinamento e o método <i>transform</i> usa esses parâmetros para transformar os dados. Qualquer matriz de dados a ser transformada precisa ter o mesmo número de recursos que a matriz de dados que foi usada para ajustar o modelo.
<p align='center'><img src=https://www.bogotobogo.com/python/scikit-learn/images/scikit-data-processing1-missing-data/fit-transform-scikit-learn-estimator.png></p>


<p align='justify'>Os classificadores que usamos pertencem aos chamados estimadores do <i>scikit-learn</i>, com uma <i>API</i> conceitualmente muito semelhante à classe do transformador. Os estimadores têm um método de previsão, mas também podem ter um método de transformação. Como você deve se lembrar, também usamos o método de <i>fit</i> para aprender os parâmetros de um modelo quando treinamos esses estimadores para classificação. No entanto, em tarefas de aprendizado supervisionado, também fornecemos os rótulos de classe para ajustar o modelo, que podem ser usados para fazer previsões sobre novos exemplos de dados não rotulados por meio do método de previsão, conforme ilustrado na figura a seguir:</p>
<p align='center'><img src=https://www.bogotobogo.com/python/scikit-learn/images/scikit-data-processing1-missing-data/scikit-learn-predict-method.png></p>

<h4>Como lidar com dados categóricos</h4>
<p align='justify'>Até agora, trabalhamos apenas com valores numéricos. No entanto, não é incomum que conjuntos de dados do mundo real contenham uma ou mais colunas de recursos categóricos. Agora, faremos uso de exemplos simples, mas eficazes, para ver como lidar com esse tipo de dados em bibliotecas de computação numérica. Quando estamos falando de dados categóricos, temos que distinguir ainda mais entre características <b>ordinais e nominais</b>. Os recursos ordinais podem ser entendidos como valores categóricos que podem ser classificados ou ordenados. Por exemplo, o tamanho da camiseta seria um recurso ordinal, pois podemos definir uma ordem: XL > L > M. Em contrapartida, os recursos nominais não implicam em nenhuma ordem e, para continuar com o exemplo anterior, poderíamos pensar em a cor da camiseta como um recurso nominal, pois normalmente não faz sentido dizer que, por exemplo, o vermelho é maior que o azul.</p>

In [30]:
# Criando um Conjunto de dados com classes categóricas

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [31]:
# Mapeando recursos ordinais
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [32]:
# Se quiser reverter aos valores aprendidos.
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

<h4>Codificando rótulos de classe</h4>
<p align='justify'>Muitas bibliotecas de aprendizado de máquina exigem que os rótulos de classe sejam codificados como valores inteiros. Embora a maioria dos estimadores para classificação no <i>scikit-learn</i> converta rótulos de classe em inteiros internamente. Considera-se uma boa prática fornecer rótulos de classe como matrizes de inteiros para evitar falhas técnicas. Para codificar os rótulos de classe, podemos usar uma abordagem semelhante ao mapeamento de recursos ordinais discutidos anteriormente. Precisamos lembrar que os rótulos de classe não são ordinais e não importa qual número inteiro atribuímos a um rótulo de <i>string</i> específico. Assim, podemos simplesmente enumerar os rótulos das classes, começando em 0:</p>

In [33]:
import numpy as np

# Crie um dict de mapeamento para converter rótulos de classe de strings para inteiros

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [34]:
# Agora podemos converter os rótulos para converter rótulos de classe strings para inteiros
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [35]:
# Para reverter

inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [36]:
# Alternativamente, podemos usar o LabelEncoder
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

In [37]:
# Reverter o mapeamento utilizando o método inverse_transform
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

<h4>Executando a codificação one-hot em recursos nominais</h4>
<p align='justify'>Como os estimadores do <i>scikit-learn</i> para classificação tratam rótulos de classe como dados categóricos que não implicam em nenhuma ordem (nominal), usamos o <i>LabelEncoder</i> para codificar os rótulos de <i>string</i> em inteiros.</p>

In [38]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

<p align='justify'>Se pararmos neste ponto e alimentarmos o <i>array</i> em nosso classificador, cometeremos um dos erros mais comuns ao lidar com dados categóricos. Você consegue identificar o problema? Embora os valores de cor não venham em nenhuma ordem específica, um algoritmo de aprendizado agora assumirá que verde é maior que azul e vermelho é maior que verde. Embora essa suposição esteja incorreta, o algoritmo ainda pode produzir resultados úteis. No entanto, esses resultados não seriam ideais.</i>

<p align='justify'>Para lidar com esse tipo de problema, utilizamos a técnica do <i>one-hot encoding</i>. De maneira geral, estaremos transformando cada categoria em novas <i>features</i>(colunas). Então as cores (azul, verde e vermelho) seriam transformadas em 3 colunas.</p>

In [40]:
from sklearn.preprocessing import OneHotEncoder

X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

<p align='justify'>Observe que aplicamos o <i>OneHotEncoder</i> a apenas uma única coluna, (X[:, 0].reshape(-1, 1))), para evitar modificar também as outras duas colunas na matriz. Se quisermos transformar seletivamente colunas em um array com vários recursos, podemos usar o <i>ColumnTransformer</i>, que aceita uma lista de tuplas (nome, transformador, coluna(s)) da seguinte forma:</p>

In [47]:
from sklearn.compose import ColumnTransformer

X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([ ('onehot', OneHotEncoder(), [0]), 
                               ('nothing', 'passthrough', [1, 2])]) # Usando o argumento 'passthrough' não acontece nada!!
c_transf.fit_transform(X).astype(float)

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

In [48]:
# Usando o Pandas - get_dummies()
pd.get_dummies(df[['price','color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


<p align='justify'>Quando estamos usando conjuntos de dados de codificação one-hot, temos que ter em mente que isso introduz multicolinearidade, o que pode ser um problema para certos métodos (por exemplo, métodos que exigem inversão de matriz). Se os recursos são altamente correlacionados, as matrizes são computacionalmente difíceis de inverter, o que pode levar a estimativas numericamente instáveis. Para reduzir a correlação entre as variáveis, podemos simplesmente remover uma coluna de recurso do <i>array</i> codificado <i>one-hot</i>. Observe que não perdemos nenhuma informação importante removendo uma coluna de recurso; por exemplo, se removermos a coluna color_blue, as informações do recurso ainda serão preservadas, pois se observarmos color_green=0 e color_red=0, isso implica que a observação deve ser azul.</p>

In [50]:
# Usando o argumento drop_first=True
pd.get_dummies(df[['price','color','size']],
                    drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


In [51]:
# No OneHotEncoder utilizando o drop="first" e o categories='auto'

color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([ ('onehot', color_ohe, [0]),
                               ('nothing', 'passthrough', [1, 2])])
c_transf.fit_transform(X).astype(float)

array([[ 1. ,  0. ,  1. , 10.1],
       [ 0. ,  1. ,  2. , 13.5],
       [ 0. ,  0. ,  3. , 15.3]])

<h4>Como particionar um conjunto de dados em conjuntos de dados de treinamento e teste separados</h4>
<p align='justify'>Lembre-se de que comparar previsões com rótulos verdadeiros no conjunto de teste pode ser entendido como a avaliação de desempenho imparcial de nosso modelo antes de deixá-lo solto no mundo real. Vamos preparar um novo conjunto de dados, o conjunto de dados <i>Wine</i>. Depois de pré-processarmos o conjunto de dados, exploraremos diferentes técnicas de seleção de recursos para reduzir a dimensionalidade de um conjunto de dados.</p>

In [54]:
# Carregando os dados
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
                      header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

print('Class labels: ', np.unique(df_wine['Class label']))
df_wine.head()

Class labels:  [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [55]:
from sklearn.model_selection import train_test_split

# Particionando em conjuntos de treino e teste 
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.3,     # Separando 30% para testes
                     random_state=0, 
                     stratify=y)        # Assegura que as classes de treino e teste terão classe proporcionais com base no original.

<h4>Trazendo recursos para a mesma escala</h4>
<p align='justify'>O dimensionamento de recursos é uma etapa crucial em nosso <i>pipeline</i> de pré-processamento que pode ser facilmente esquecido. Árvores de decisão e florestas aleatórias são dois dos poucos algoritmos de aprendizado de máquina em que não precisamos nos preocupar com o dimensionamento de recursos. Esses algoritmos são invariantes de escala. No entanto, a maioria dos algoritmos de aprendizado de máquina e otimização se comportam muito melhor se os recursos estiverem na mesma escala, quando implementamos o algoritmo de otimização de gradiente descendente.<p>
<p align='justify'>A importância do dimensionamento de recursos pode ser ilustrada por um exemplo simples. Vamos supor que temos dois recursos em que um recurso é medido em uma escala de 1 a 10 e o segundo recurso é medido em uma escala de 1 a 100.000, respectivamente.</p>

<p align='justify'>Agora, existem duas abordagens comuns para trazer diferentes recursos para a mesma escala: <b>normalização e padronização</b>. Esses termos são frequentemente usados ​​de forma bastante vaga em diferentes campos e o significado deve ser derivado do contexto. Na maioria das vezes, a normalização refere-se ao redimensionamento dos recursos para um intervalo de [0, 1], que é um caso especial de dimensionamento mínimo-máximo. Para normalizar nossos dados, podemos simplesmente aplicar a escala <i>min-max</i> a cada coluna de recurso, onde o novo valor, de um exemplo, pode ser calculado da seguinte forma:

In [59]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

<p align='justify'>Embora a normalização por meio do dimensionamento <i>min-max</i> seja uma técnica comumente usada que é útil quando precisamos de valores em um intervalo limitado, a padronização pode ser mais prática para muitos algoritmos de aprendizado de máquina, especialmente para algoritmos de otimização, como gradiente descendente.</p>
<p align='justify'>A razão é que muitos modelos lineares, como a regressão logística e SVM, inicializam os pesos em 0 ou pequenos valores aleatórios próximos a 0. Usando padronização, centralizamos o recurso colunas em média 0 com desvio padrão 1 para que as colunas de recursos tenham os mesmos parâmetros de uma distribuição normal padrão (média zero e variância unitária), o que facilita o aprendizado dos pesos. Além disso, a padronização mantém informações úteis sobre valores discrepantes e torna o algoritmo menos sensível a eles em contraste com o dimensionamento mínimo-máximo, que dimensiona os dados para um intervalo limitado de valores.</p>

In [60]:
# Implementando a Padronização
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [66]:
ex = np.array([0, 1, 2, 3, 4, 5])

print('Standardized:', (ex - ex.mean()) / ex.std())

# Please note that pandas uses ddof=1 (sample standard deviation) 
# by default, whereas NumPy's std method and the StandardScaler
# uses ddof=0 (population standard deviation)

# normalize
print('  Normalized:', (ex - ex.min()) / (ex.max() - ex.min()))

Standardized: [-1.46385011 -0.87831007 -0.29277002  0.29277002  0.87831007  1.46385011]
  Normalized: [0.  0.2 0.4 0.6 0.8 1. ]


<p align='justify'>Novamente, também é importante destacar que ajustamos a classe <i>StandardScaler</i> apenas uma vez – nos dados de treinamento – e usamos esses parâmetros para transformar o conjunto de dados de teste ou qualquer novo ponto de dados.</p>
<p align='justify'>Outros métodos mais avançados para dimensionamento de recursos estão disponíveis no <i>scikit-learn</i>, como o <i>RobustScaler</i>. O <i>RobustScaler</i> é especialmente útil e recomendado se estivermos trabalhando com pequenos conjuntos de dados que contêm <b>muitos valores discrepantes</b>. Da mesma forma, se o algoritmo de aprendizado de máquina aplicado a esse conjunto de dados for propenso a <i>overfitting</i>, o <i>RobustScaler</i> pode ser uma boa escolha. Operando em cada coluna de recurso independentemente, o <i>RobustScaler</i> remove o valor mediano e dimensiona o conjunto de dados de acordo com o 1º e o 3º quartil do conjunto de dados (ou seja, o 25º e o 75º quantil, respectivamente) de modo que valores mais extremos e valores discrepantes se tornem menos pronunciados.</p>

<h4>Selecionando recursos significativos</h4>